# Plotting Using MongoDB, Plotly, and Pandas python

In [36]:
import pymongo
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

### Connection String and setting Pandas to use plotly

In [37]:
CONN_STRING = 'mongodb+srv://<user:pass>@<clusterURL>'
pd.options.plotting.backend = "plotly"

### Connecting to a collection through the MongoClient interface

In [38]:
client = pymongo.MongoClient(CONN_STRING)
db = client.get_database('NCIDCTD')

### Making an Aggregation Pipeline for Specific NSC on the old OneDose collection

In [117]:
coll = db.get_collection('onedose_tline')
data = coll.aggregate([
   # {
    #    '$limit': 1000
    #},
    {
        '$unwind': {
            'path': '$tline', 
            'preserveNullAndEmptyArrays': False
        }
    }, {
        '$match': {
            'expid':'9108NP24',
            'tline.nsc':386
        }
    }, {
        '$project': {
            '_id': 0, 
            'tline.nsc': 1, 
            'tline.cellnbr': 1, 
            'tline.panelnbr': 1,
            'tline.giprcnt':1
        }
    }
])

### Cleaing and reshaping the data a bit.
##### The Cell_Lines are still normalized

In [118]:
df = pd.DataFrame([d['tline'] for d in data])
df = df.sort_values('giprcnt',ascending=False,ignore_index=True)
df = df.dropna()
df['cell_line'] = df['panelnbr'].astype(str)+','+df['cellnbr'].astype(str)

### Creating a basic Plotly Line graph showing Growth Perctg per Cell Line

In [121]:
fig1 = px.line(x=df.cell_line,y=df.giprcnt,title='NSC 386, Exp 9108NP24',labels={'x':'Cell Line','y':'Growth Percentage'})

In [122]:
fig1.show()

*The new one dose collection will make graphing significantly easier as cell and panel names are denormalized and leaner*

## Creating OneDose Waterfall Graph with New OneDose based on ExpID

In [75]:
coll = db.get_collection('new_onedose')

In [99]:
data = coll.aggregate([
    {
        '$match': {
            'expid': '0701OS80'
        }
    }, {
        '$project': {
            '_id': 0, 
            'tline': 1
        }
    }, {
        '$unwind': {
            'path': '$tline', 
            'preserveNullAndEmptyArrays': False
        }
    }
])
exp_df = pd.DataFrame([d['tline'] for d in data])
exp_df.GrowthPercent = exp_df.GrowthPercent.apply(lambda x: x['Average'])
exp_df = exp_df.sort_values('GrowthPercent',ascending=False,ignore_index=True)

### Cleaning data by using NSC and isolating down to 3 variables

In [100]:
wf_data = exp_df[exp_df['Nsc'] == 123127]
wf_data = wf_data[['GrowthPercent','cellpnl','cellline']]
wf_data['Panel'] = [p['panelnme'] for p in wf_data.cellpnl]
wf_data['Cell'] = [c['cellname'] for c in wf_data.cellline]
wf_data.drop(columns=['cellpnl','cellline'], inplace=True)


### Use a Horizontal Bar graph to display the data grouped by Panel

In [101]:
bar1 = px.bar(wf_data,
              x="GrowthPercent",
              y="Cell",
              labels={"GrowthPercent":"Growth Inhibition %","Cell":"Cell Line"},
              barmode='group',color='Panel',
              orientation='h',range_x=[100,-100],
              height=1000,
              title="GI% For 123127"
              
             )
#bar1.update_layout(bargroupgap=0,bargap=0)
bar1.update_traces(width=1)
bar1.update_layout(title_x=0.5)
bar1.show()


## Apparently Waterfall is not correct.

**Waterfall Chart:**
>A waterfall chart is a form of data visualization that helps in understanding the cumulative effect of sequentially introduced positive or negative values. These intermediate values can either be time based or category based.

In [102]:
wf_data = exp_df[exp_df['Nsc'] == 123127]
wf_data = wf_data[['GrowthPercent','cellpnl','cellline']]
wf_data['Panel'] = [p['panelnme'] for p in wf_data.cellpnl]
wf_data['Cell'] = [c['cellname'] for c in wf_data.cellline]

In [103]:
txt="""
    Cell Panel: {}\n
    Cell Line: {}\n
    Average Growth Inhibition: {}    
"""
fig2 = go.Figure(go.Waterfall(
    name="NSC 123127",orientation='v',
    x=[cl['cellname'] for cl in wf_data['cellline']],
    y=[w_d for w_d in wf_data['GrowthPercent']],
    text=[txt.format(w_d.cellpnl['panelnme'],w_d.cellline['cellname'],w_d.GrowthPercent) for w_d in wf_data.itertuples()],
    connector = {"mode":"spanning", "line":{"width":1, "color":"rgb(0, 0, 0)", "dash":"solid"}}
))

fig2.update_layout(title="NSC 123127")

What we actually want is just a horizontal bar chart in sorted order.

## 5 Dose Conc Response Curve Attempt 1

Using NSC Data derived from the fivedose_panel collection, display a response curve line graph.

In [49]:
coll = db.get_collection('fivedose_panel')
data = [d for d in 
                    (coll.aggregate(
                        [                            
                            {
                                '$match': {
                                    'expid': '0001RM03'
                                }
                            }, {
                                '$project': {
                                    'paneldetails_1': 1, 
                                    'paneldetails_2': 1, 
                                    'paneldetails_3': 1, 
                                    'paneldetails_4': 1, 
                                    'paneldetails_5': 1, 
                                    '_id': 0
                                }
                            }

                        ])
                    )
                  ]
dfs = []
for item in data:
    for stuff in item:
        for cols in item[stuff]:
            dfs.append(pd.Series(cols))
df = pd.DataFrame(dfs)


In [50]:
df[df['nsc'] == 123127]

,plandate,testseq,prefix,nsc,hiconc,conc,NCI-H23,NCI-H522,A549/ATCC,ekvx,...,m14,UACC-62,UACC-257,DU-145,SNB-19,SNB-75,u251,SF-268,SF-295,SF-539
0,2000-01-11T00:00:00,1,S,123127,0.000025,2.500000e-05,-80,-98,12,-7,...,-72,-91,-62,10,-8,-65,-11,-66,-24,-80
2,2000-01-11T00:00:00,1,S,123127,0.000025,2.500000e-06,-40,-85,-42,27,...,-38,-93,-30,-15,14,-51,5,-35,-25,-33
4,2000-01-11T00:00:00,1,S,123127,0.000025,2.500000e-07,38,-40,54,92,...,59,25,46,28,15,-18,17,6,33,32
6,2000-01-11T00:00:00,1,S,123127,0.000025,2.500000e-08,89,5,71,96,...,100,76,92,83,79,47,74,73,86,73
8,2000-01-11T00:00:00,1,S,123127,0.000025,2.500000e-09,97,59,85,102,...,104,92,98,98,87,89,90,91,106,86


In [51]:
nsc = '123127'

### Data cleaning, transformations
First we retrieve by NSC.  Then, we remove columns not used.  Then, we must uppercase them and remove double white spaces with single white space.

In [52]:

nsc_df = df.query(f"nsc == {nsc}").sort_values(by="conc")
nsc_df = nsc_df.drop(['plandate','testseq','prefix','nsc','hiconc'],axis=1)
nsc_df.columns = [x.upper().replace('  ',' ') for x in nsc_df.columns]
nsc_df = nsc_df.set_index('CONC')
nsc_df = nsc_df.dropna(axis=1) # remove cell lines that were not tested fully or at all
nsc_df

,NCI-H23,NCI-H522,A549/ATCC,EKVX,NCI-H226,NCI-H322M,NCI-H460,HOP-62,HOP-92,HT29,...,M14,UACC-62,UACC-257,DU-145,SNB-19,SNB-75,U251,SF-268,SF-295,SF-539
CONC,,,,,,,,,,,,,,,,,,,,,
2.500000e-09,97,59,85,102,84,99,56,91,100,103,...,104,92,98,98,87,89,90,91,106,86
2.500000e-08,89,5,71,96,79,97,40,85,69,94,...,100,76,92,83,79,47,74,73,86,73
2.500000e-07,38,-40,54,92,27,75,10,35,33,52,...,59,25,46,28,15,-18,17,6,33,32
2.500000e-06,-40,-85,-42,27,-14,18,-8,-15,-6,11,...,-38,-93,-30,-15,14,-51,5,-35,-25,-33
2.500000e-05,-80,-98,12,-7,-38,-8,-32,-43,-44,3,...,-72,-91,-62,10,-8,-65,-11,-66,-24,-80


In [53]:
conc_resp_fig = go.Figure()
for cell in nsc_df.columns:
    conc_resp_fig.add_trace(
        go.Scatter(
            x=nsc_df[cell].index,
            y=nsc_df[cell].values,
            mode='lines+markers',
            name=f'{cell}',
            line_shape='spline',
        ))
conc_resp_fig.update_xaxes(type='log',dtick='log', exponentformat='E',title='Concentration (mol)')
conc_resp_fig.update_yaxes(title='Growth Inhibition Pct (GI%)')
conc_resp_fig.update_layout(height=700,title=f'NSC-{nsc} 5-Concentration Response',title_x=.44, legend_title_text='Cell Lines',)
conc_resp_fig.show()

# Trying to use Dash or Tiled Conc resp curve charts by panel

In [54]:
import os
import oracledb
import sys
oracledb.version = "8.3.0"
sys.modules["cx_Oracle"] = oracledb
from sqlalchemy import create_engine

### Define the joining function

In [55]:
def join_panel(val,panel_map):
    for key in panel_map.keys():
        if val == key:
            return panel_map[key]
    return 'not found'

### Define function to reshape exp data

In [56]:
def prep_data(df,nsc):
    nsc_df = df.query(f"nsc == {nsc}").sort_values(by="conc")
    nsc_df = nsc_df.drop(['plandate','testseq','prefix','nsc','hiconc'],axis=1)
    nsc_df.columns = [x.upper().replace('  ',' ').replace('MDA-N','HS 578T') for x in nsc_df.columns]
    nsc_df = nsc_df.set_index('CONC')
    nsc_df = nsc_df.dropna(axis=1) # remove cell lines that were not tested fully or at all
    return nsc_df

### Define function to create dictionary grouped by panel

In [57]:
def create_grouped_data_dict(df):
    data_dict = dict()
    for panel in df_nci60['panel'].unique():
        data_dict[panel] = pd.DataFrame(index=df.index)
    
    for col in df.columns: # Column is cell line
        panel_name = df_nci60[df_nci60['cellname'] == col]['panel'].iloc[0]
        data_dict[panel_name] = pd.concat([data_dict[panel_name],df[col]],axis=1).dropna(axis=1)
    return data_dict
        

### Define function to create a Plotly figure

In [58]:
def create_conc_resp_plot(df):
    conc_resp_fig = go.Figure()
    for cell in nsc_df.columns:
        conc_resp_fig.add_trace(
            go.Scatter(
                x=nsc_df[cell].index,
                y=nsc_df[cell].values,
                mode='lines+markers',
                name=f'{cell}',
                line_shape='spline',
            ))
    conc_resp_fig.update_xaxes(type='log',dtick='log', exponentformat='E',title='Concentration (mol)')
    conc_resp_fig.update_yaxes(title='Growth Inhibition Pct (GI%)')
    conc_resp_fig.update_layout(height=700,title=f'NSC-{nsc} 5-Concentration Response',title_x=.44, legend_title_text='Cell Lines',)
    return conc_resp_fig

### Define a Map for Panel Codes

In [59]:
panel_name_map = {"REN":"Renal Cancer","MEL":"Melanoma","PRO":"Prostate Cancer","CNS":"CNS Cancer","LEU":"Leukemia","OVA":"Ovarian Cancer","COL":"Colon Cancer","BRE":"Breast Cancer","LNS":"Non-Small Cell Lung Cancer"}

### Need to connect to Oracle to get the Cell names and such

In [60]:
username = '<oracle Username>'
pw = '<oracle PW>'
host = '<Oracle Host URL>'
service = '<Oracle Service>'
engine = create_engine(f'oracle://{username}:{pw}@',
                       connect_args={
                           "host": host,
                           "port": 1521,
                           "service_name": service
                       }
         )

### Make the call to oracle and then create column with joining function

In [61]:
nci60_sql = 'SELECT CELLNAME, PANELCDE FROM COMMON.cellline WHERE COMPARE > 0'
df_nci60 = pd.read_sql(nci60_sql, engine)
df_nci60['panel'] = [join_panel(x,panel_name_map) for x in df_nci60.panelcde]

## Driver Section
First, we need to create a dictionary of DataFrames grouped by Panel.  Then, we can generate the set of graphs.

In [62]:
nsc = '123127'
nsc_df = prep_data(df,nsc)
data_dict = create_grouped_data_dict(nsc_df)
data_dict

{'Renal Cancer':               UO-31  SN12C  A498  CAKI-1  RXF 393  786-0  ACHN  TK-10
 CONC                                                                 
 2.500000e-09    111     96    78     101       93     99    96     85
 2.500000e-08    103     89    73     109       87     89    92     78
 2.500000e-07     83     53    57      94       59     40    52     48
 2.500000e-06     14     40   -55      39      -14     24     6    -13
 2.500000e-05     -4     27   -74       2      -63     19    -8    -16,
 'Melanoma':               MDA-MB-435  LOX IMVI  SK-MEL-2  SK-MEL-5  SK-MEL-28  M14  \
 CONC                                                                     
 2.500000e-09          98        90       108        95        103  104   
 2.500000e-08          95        65       106        82         95  100   
 2.500000e-07          56        18        51        40         61   59   
 2.500000e-06         -65         2       -60       -87        -39  -38   
 2.500000e-05         -5

## ------- Scratch Pad Below ---------

In [63]:
t = df_nci60['panel']

In [64]:
t.unique()

array(['Renal Cancer', 'Melanoma', 'Prostate Cancer', 'CNS Cancer',
       'Leukemia', 'Ovarian Cancer', 'Colon Cancer', 'Breast Cancer',
       'Non-Small Cell Lung Cancer'], dtype=object)

In [65]:
nsc_df

,NCI-H23,NCI-H522,A549/ATCC,EKVX,NCI-H226,NCI-H322M,NCI-H460,HOP-62,HOP-92,HT29,...,M14,UACC-62,UACC-257,DU-145,SNB-19,SNB-75,U251,SF-268,SF-295,SF-539
CONC,,,,,,,,,,,,,,,,,,,,,
2.500000e-09,97,59,85,102,84,99,56,91,100,103,...,104,92,98,98,87,89,90,91,106,86
2.500000e-08,89,5,71,96,79,97,40,85,69,94,...,100,76,92,83,79,47,74,73,86,73
2.500000e-07,38,-40,54,92,27,75,10,35,33,52,...,59,25,46,28,15,-18,17,6,33,32
2.500000e-06,-40,-85,-42,27,-14,18,-8,-15,-6,11,...,-38,-93,-30,-15,14,-51,5,-35,-25,-33
2.500000e-05,-80,-98,12,-7,-38,-8,-32,-43,-44,3,...,-72,-91,-62,10,-8,-65,-11,-66,-24,-80


In [66]:
#create_grouped_data_dict(nsc_df)
for col in nsc_df.columns:
    try:
        print(df_nci60[df_nci60['cellname'] == col]['panel'].iloc[0])
    except:
        print('error: '+col)
    


Non-Small Cell Lung Cancer
Non-Small Cell Lung Cancer
Non-Small Cell Lung Cancer
Non-Small Cell Lung Cancer
Non-Small Cell Lung Cancer
Non-Small Cell Lung Cancer
Non-Small Cell Lung Cancer
Non-Small Cell Lung Cancer
Non-Small Cell Lung Cancer
Colon Cancer
Colon Cancer
Colon Cancer
Colon Cancer
Colon Cancer
Colon Cancer
Colon Cancer
Breast Cancer
Ovarian Cancer
Breast Cancer
Melanoma
Breast Cancer
Breast Cancer
Breast Cancer
Ovarian Cancer
Ovarian Cancer
Ovarian Cancer
Ovarian Cancer
Ovarian Cancer
Ovarian Cancer
Leukemia
Leukemia
Leukemia
Leukemia
Leukemia
Leukemia
Renal Cancer
Renal Cancer
Renal Cancer
Renal Cancer
Renal Cancer
Renal Cancer
Renal Cancer
Renal Cancer
Melanoma
Melanoma
Melanoma
Melanoma
Melanoma
Melanoma
Melanoma
Prostate Cancer
CNS Cancer
CNS Cancer
CNS Cancer
CNS Cancer
CNS Cancer
CNS Cancer


In [67]:
df_nci60[df_nci60['panel'] == 'Breast Cancer']

,cellname,panelcde,panel
27,MDA-MB-231/ATCC,BRE,Breast Cancer
44,MDA-MB-468,BRE,Breast Cancer
46,BT-549,BRE,Breast Cancer
47,T-47D,BRE,Breast Cancer
56,MCF7,BRE,Breast Cancer
58,HS 578T,BRE,Breast Cancer


In [68]:
data = coll.aggregate([
            {
                '$match': {
                    'expid': '0001LS03'
                }
            }, {
                '$project': {
                    '_id': 0, 
                    'tline': 1
                }
            }, {
                '$unwind': {
                    'path': '$tline', 
                    'preserveNullAndEmptyArrays': False
                }
            }, {
                '$match': {
                    'tline.nsc': 123127
                }
            }
        ])
df = pd.DataFrame([d['tline'] for d in data])

In [69]:
df.columns

Index(['testseq', 'panelnbr', 'cellnbr', 'inocdens', 'prefix', 'nsc',
       'presampl', 'sample', 'fract', 'concunit', 'dilufact', 'solind',
       'vehicle', 'hiconc', 'loconc', 'screener', 'barcode', 'dosernbr',
       'status', 'doseqc', 'ic50ind1', 'ic90ind1', 'ic100in1', 'ic50ind2',
       'ic90ind2', 'ic100in2', 'gi50ind1', 'gi50ind2', 'tgiind1', 'tgiind2',
       'lc50ind1', 'lc50ind2', 'lc100in1', 'lc100in2', 'ic50', 'ic90', 'ic100',
       'gi50', 'tgi', 'lc50', 'lc100'],
      dtype='object')

In [70]:
gi50 = df[['panelnbr','cellnbr','gi50','tgi','lc50']].sort_values(['panelnbr','cellnbr','gi50'])
tgi = df[['panelnbr','cellnbr','gi50','tgi','lc50']].sort_values(['panelnbr','cellnbr','tgi'])
lc50 = df[['panelnbr','cellnbr','gi50','tgi','lc50']].sort_values(['panelnbr','cellnbr','lc50'])